**This notebook I used in order to accomplish my Mini challenge task**

**I especially kept all attempts to build it**

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.datasets import ImageFolder


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

class CelebFaceDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.file_names = self.df['File Name'].tolist()
        self.categories = pd.factorize(self.df['Category'])[0]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.file_names[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.categories[idx], dtype=torch.long)
        return image, label

train_dataset = CelebFaceDataset('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/train_small.csv', '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/train_small', transform=data_transforms['train'])
val_dataset = CelebFaceDataset('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/train_small.csv', '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/train_small', transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

resnet = models.resnet18(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, len(train_dataset.categories))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

'''
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/weights.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

'''

num_epochs = 14
for epoch in range(num_epochs):
    running_loss = 0.0
    resnet.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1} loss: {running_loss / len(train_loader):.3f}')

    resnet.eval()
    val_acc = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet(inputs)
            _, preds = torch.max(outputs, 1)
            val_acc += torch.sum(preds == labels.data)
    val_acc = val_acc.double() / len(val_dataset)
    print(f'Validation accuracy: {val_acc:.4f}')

torch.save({
            'epoch': epoch,
            'model_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/weights.pth')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can als

Epoch 1 loss: 4.515


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Validation accuracy: 0.0548


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Validation accuracy: 0.9409


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 14 loss: 0.255


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Validation accuracy: 0.9422


In [ ]:
"""
test_dir = '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/tesst/Photos'
test_dataset = ImageFolder(test_dir, transform=data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/weights.pth')
resnet.load_state_dict(checkpoint['model_state_dict'])

# Make predictions on the test set
predictions = []
with torch.no_grad():
  for inputs, _ in test_loader:
    inputs = inputs.to(device)
    outputs = resnet(inputs)
    _, preds = torch.max(outputs, 1)
predictions.extend(preds.cpu().numpy())

# Create a mapping from category IDs to category names
category_map = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/category.csv').set_index('Id')['Category'].to_dict()

# Save predictions to CSV file
submission = pd.DataFrame({'Id': range(len(predictions)), 'Category': [category_map[pred] for pred in predictions]})
submission.to_csv('submission.csv', index=False)



"""

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
import zipfile
import requests

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.Lambda(lambda img: img.convert('RGB')),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Lambda(lambda img: img.convert('RGB')),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

class CelebFaceDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

        self.category_to_idx = {category: idx for idx, category in enumerate(sorted(set(self.df['Category'])))}

        self.file_names = self.df['File Name'].tolist()
        self.categories = [self.category_to_idx[category] for category in self.df['Category']]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        img_path = os.path.join(self.img_dir, file_name)
        image = Image.open(img_path)
        label = self.categories[idx]
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.long)
        return image, label


train_dataset = CelebFaceDataset('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/train_small.csv', '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/train_small', transform=data_transforms['train'])
val_dataset = CelebFaceDataset('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/train_small.csv', '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/train_small', transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, len(train_dataset.categories))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1} loss: {running_loss / len(train_loader):.3f}')

resnet.eval()
val_acc = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet(inputs)
        _, preds = torch.max(outputs, 1)
        val_acc += torch.sum(preds == labels.data)
val_acc = val_acc.double() / len(val_dataset)
print(f'Validation accuracy: {val_acc:.4f}')


torch.save({
            'epoch': epoch,
            'model_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/new_weights_50res.pth')

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 140MB/s]
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1 loss: 4.876
Epoch 2 loss: 4.571
Epoch 3 loss: 4.481
Epoch 4 loss: 4.351
Epoch 5 loss: 4.276
Epoch 6 loss: 4.121
Epoch 7 loss: 4.003
Epoch 8 loss: 3.926
Epoch 9 loss: 3.845
Epoch 10 loss: 3.727
Epoch 11 loss: 3.645
Epoch 12 loss: 3.538
Epoch 13 loss: 3.415
Epoch 14 loss: 3.294
Epoch 15 loss: 3.159
Epoch 16 loss: 3.054
Epoch 17 loss: 3.126
Epoch 18 loss: 2.762
Epoch 19 loss: 2.510
Epoch 20 loss: 2.234
Validation accuracy: 0.4012


In [ ]:
from torchvision.datasets import ImageFolder
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

test_dir = '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/tesst'
test_dataset = ImageFolder(test_dir, transform=data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

resnet = models.resnet18(pretrained=False)
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/weights.pth')
resnet.load_state_dict(checkpoint['model_state_dict'])


predictions = []
with torch.no_grad():
  for inputs, _ in test_loader:
    inputs = inputs.to(device)
    outputs = resnet(inputs)
    _, preds = torch.max(outputs, 1)
predictions.extend(preds.cpu().numpy())

category_map = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/category.csv').set_index('Id')['Category'].to_dict()

submission = pd.DataFrame({'Id': range(len(predictions)), 'Category': [category_map[pred] for pred in predictions]})
submission.to_csv('submission.csv', index=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([6955, 512]) from checkpoint, the shape in current model is torch.Size([1000, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([6955]) from checkpoint, the shape in current model is torch.Size([1000]).

In [ ]:
#/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/tesst/Photos
!cd drive

In [ ]:
!ls /content/drive/MyDrive/'Colab Notebooks'/'ECE 500024'/MINI/tesst/Photos

106.jpg  131.jpg  156.jpg  172.jpg  197.jpg  40.jpg  65.jpg  81.jpg
115.jpg  140.jpg  15.jpg   181.jpg  22.jpg   47.jpg  6.jpg   90.jpg
122.jpg  147.jpg  165.jpg  190.jpg  31.jpg   56.jpg  72.jpg  97.jpg


In [ ]:
from torchvision.datasets import ImageFolder
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

test_dir = '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/tesst'
test_dataset = ImageFolder(test_dir, transform=data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load the pre-trained ResNet18 model
# resnet = models.resnet18(pretrained=False)
# num_classes = len(test_dataset.classes)
# resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

# checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/weights.pth')
# resnet.load_state_dict(checkpoint['model_state_dict'])


resnet = models.resnet18(pretrained=False)
num_classes = len(test_dataset.classes)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/weights.pth')
resnet.load_state_dict(checkpoint['model_state_dict'], strict=False)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet.to(device)

predictions = []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = resnet(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

category_map = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/category.csv').set_index('Id')['Category'].to_dict()

submission = pd.DataFrame({'Id': range(len(predictions)), 'Category': [category_map[pred] for pred in predictions]})
submission.to_csv('submission.csv', index=False)

RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([6955, 512]) from checkpoint, the shape in current model is torch.Size([1, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([6955]) from checkpoint, the shape in current model is torch.Size([1]).

In [ ]:
from torchvision.datasets import ImageFolder
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

test_dir = '/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/my_test'
test_dataset = ImageFolder(test_dir, transform=data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

resnet = models.resnet50(pretrained=False)

checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/new_weights_50res.pth')

num_classes = checkpoint['model_state_dict']['fc.weight'].size(0)

resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

resnet.load_state_dict(checkpoint['model_state_dict'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet.to(device)

predictions = []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = resnet(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

# category_map = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/category.csv').set_index('Id')['Category'].to_dict()

category_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ECE 500024/MINI/category.csv', index_col=0)

category_map = category_df['Category'].to_dict()


submission = pd.DataFrame({'Id': range(len(predictions)), 'Category': [category_map[pred] for pred in predictions]})
submission.to_csv('submission.csv', index=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
